# Recipe Generation with GPT-2


This notebook explores how to **fine-tune a transformer** for a custom text completion task -- specifically, we'll teach GPT-2 to generate full recipe instructions from just a list of ingredients.

Before the world met ChatGPT, GPT-2 was already showing us what generative AI could do -- if we were paying attention. To learn more about this large language model, which was **trained on 8 million web pages**, check out the official model card on [Hugging Face](https://huggingface.co/gpt2).

Before we get cooking, here's a roadmap of what is to come:
- Set up the GPU runtime
- Download and inspect the dataset
- Create a DataFrame and split it into training and test sets
- Test the base model on a single sentence
- Fine-tune the model using the recipe dataset
- Evaluate the fine-tuned model's recipe-writing skills

## GPU time

In [ ]:
# Check which GPU is available for model training
!nvidia-smi -L

In [ ]:
# Install, if needed, and import the Hugging Face Transformers library
!pip install transformers
import transformers

## Download the dataset

In [ ]:
import pandas as pd

In [ ]:
# Install gdown, a Python tool that helps you download files from Google Drive directly into your Colab
!pip install --upgrade gdown

This dataset includes 120K recipes! All the recipes are formatted in the same way, and, as we will see, formatting is critical for solving text completion tasks like this. Part of the magic of transformers is that they are able to recognize and learn patterns.

In [ ]:
import gdown
# Download the recipe dataset (120K examples) from Google Drive
gdrivelink='https://drive.google.com/uc?id=10KF1LqW9k2MgTb1GwSPlfvX1INFPDxi5'
gdown.download(gdrivelink, quiet=False)

Recipe dataset at  https://eightportions.com/datasets/Recipes/

## Create the DataFrame

In [ ]:
# Create the DataFrame that we just downloaded
df = pd.read_csv('recipes.csv')

In [ ]:
# Check the first five rows
df.head()

In [ ]:
# Check the lenght
len(df)

In [ ]:
# Now check the shape
df.shape

In [ ]:
# Check for null values
df.isna().sum()

Time to explore the `Combined` column -- the only column that matters in this text generation task that we are going to have GPT-2 complete.

In [ ]:
# 'Combined' is the only column we care about in this exercise
# It's the combined ingredients and instructions and it's essential that we format the training set vert carefully
df.combined

In [ ]:
# Let's improve the formatting by printing the column and selecting .iloc[0], or just the first row
# Important note: So, this is the format for every element in this column
print(df.combined.iloc[0])

Notice: Every set of Instructions ends with the following -- <|endoftext|>. This is crucial because the model needs to know where in the training set a sentence ends.

Also, notice that ingredients in the text often appear in the same order in the Ingredients section *and* the Instructions section. Humans can notice this pattern, of course, and transformers can, too.

So, here's **the gameplan**: We will show GPT-2 120K examples where the Ingredients section is followed by the Instructions section. Then we will provide it just the Ingredients section and it will be prompted to generate text, or Instructions.

## Make the training and test set

So, how do you measure the performance of generating text?

It's a little murky.

Even if there is not a clear metric we can use, we will still split the dataset into two -- training and test.

In [ ]:
# Create the training set, just the 'combined' column values and grab the first 120,000
dataset_train = df.combined.values[:120000]

In [ ]:
# Check the length
len(dataset_train)

In [ ]:
# For the test set, we will take the rest
dataset_test = df.combined.values[120000:]

In [ ]:
# Check the length
len(dataset_test)

As we see above, we have a huge training set. But we need to turn the training and text sets into **text files**. And this is why those <|endoftext|> tokens are so important.

In [ ]:
# Here's how to create the text file for 'dataset_train'
with open('dataset_train.txt', 'w') as f:
  f.write('\n'.join(dataset_train))

In [ ]:
# And now for the test set
with open('dataset_test.txt', 'w') as f:
  f.write('\n'.join(dataset_test))

In [ ]:
# These are massive files, so, let's just display a small piece of 'dataset_train.txt'
!head -20 dataset_train.txt

## Load the pretrained GPT-2 model

We're now ready to load the core ingredients: the **pretrained GPT-2 model** and its **tokenizer**. We'll use Hugging Face's `transformers` library to grab both.

Specifically, we’ll be using:
1. `AutoModelForCausalLM` -- the GPT-2 language model for text generation
2. `AutoTokenizer` -- the tokenizer that breaks text into tokens the model understands

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Here you can either use your own model or download one from Hugging Face
# We, of course, will be using GPT-2 but note: The distiled version, which is small, fast
# and still almost as accurate as the famous original
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

## Alternatively, load a fine-tuned model we previously saved

In [ ]:
import gdown
gdrivelink='https://drive.google.com/drive/folders/1qHEQ6zpOeGDiBlZ3q4zkpMVSVa86d69q?usp=sharing'
gdown.download_folder(gdrivelink, quiet=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('recipe_generation_model')
model = AutoModelForCausalLM.from_pretrained('recipe_generation_model')

## Test the model on one sentence


Let's find out how the pretrained model does on the task of completing a sentence.

We’ll first encode a sentence using the tokenizer and see what the model sees.

**Side Note**: When we use `return_tensors='pt'`, we're actually pulling in **PyTorch** under the hood. Even though we didn’t explicitly import it, Hugging Face’s `transformers` library is built on top of PyTorch by default.
>
So, the output is a PyTorch tensor  -- a powerful data structure that we’ll work with more as we go deeper into model building.

In [ ]:
# First, define the input text
# And then encode it with 'tokenizer.encode()'
input_text = 'I hit the slopes early to snowboard, caught the first lift, and by noon I had already'
enc_input = tokenizer.encode(input_text, return_tensors='pt', add_special_tokens=False)
enc_input

Now that we've tokenized our input sentence, we can feed it into the model and generate predictions to complete the sentence.

Note: This block of code is a **beast** so make sure to read through those comment lines.

In [ ]:
output_sequences = model.generate(
    input_ids = enc_input, # The encoded input from earlier
    max_length= 70,  # The max length of the generated sentence
    temperature = 0.9, # Controls randomness, closer to 1 = more creative, closer to 0 = more predictable
    top_k = 20, # Considers only the top 20 most likely next words
    top_p = 0.9, # Allows dynamic cutoff of word options based on cumulative probabilioty
    repetition_penalty = 1, # Penalty for repeating a word in the input
    do_sample = True, # If true, this allows for randomness instead of always picking the highest-probability word
    num_return_sequences = 5 # Number of output sentences
)
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

Obviously, these outputs are -- mostly, if not enitrely -- nonsense. But what about a recipe? Can are model accomplish that?

In [ ]:
# Display the first element from 'dataset_train'
dataset_train[0]

In [ ]:
# Need to split the above output and add that 'Instructions' column as a prompt
dataset_train[0].split('Instructions:')[0]+'Instructions:'

In [ ]:
# Now, let's try it as our new input text
# Note: Need to increase the length of the generated sentence
new_input_text = dataset_train[0].split('Instructions:')[0]+'Instructions:'
enc_input = tokenizer.encode(new_input_text, return_tensors='pt', add_special_tokens=False)
enc_input
output_sequences = model.generate(
    input_ids = enc_input, # The encoded input from earlier
    max_length= 150,  # The max length of the generated sentence
    temperature = 0.9, # Controls randomness, closer to 1 = more creative, closer to 0 = more predictable
    top_k = 20, # Considers only the top 20 most likely next words
    top_p = 0.9, # Allows dynamic cutoff of word options based on cumulative probabilioty
    repetition_penalty = 1, # Penalty for repeating a word in the input
    do_sample = True, # If true, this allows for randomness instead of always picking the highest-probability word
    num_return_sequences = 5 # Number of output sentences
)
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

The results aren't a total mess -- well, at least the ones that aren't left blank.

GPT-2 is speaking gramatically correct English but still isn't accomplishing the task we set out to do. So, up next? Fine-tuning the model.

# Fine-tune the model

To begin fine-tuning, we’ll download a lightweight training script provided by Michele Samorani, Associate Professor, Information Systems & Analytics at the Leavey School of Business at Santa Clara Univeristy.

The script is adapted from Hugging Face’s official examples. it handles the model training loop and saves outputs to a local `experiments/` folder.

The original Hugging Face version can be found [here](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py).



In [ ]:
# Download the custom fine-tuning script from the course server
!curl https://webpages.scu.edu/ftp/msamorani/NLP/run_lm_finetuning.py > run_lm_finetuning.py

In [ ]:
# Create a folder to store training outputs and model checkpoints
!mkdir experiments

In [ ]:
# Set a couple of parameters
# First, epochs set to three so the model will go through the dataset three times
epochs = 3

In [ ]:
# Then find the file that has our training set
file_with_training_set = 'dataset_train.txt'

In [ ]:
# Create the bash scipt to fine-tune the language model
# This loops through the number of epochs and build a command that:
# 1. Calls the training script
# 2. Points to the input file
# 3. Saves model checkpoints to a directory named by epoch
# 4. Uses distilGPT2 and basic Hugging Face flags
text = f"for epoch in {epochs} \n"+\
"do \n"+\
"python run_lm_finetuning.py "+\
f"--output_dir=experiments/epoch_{epochs} "+\
"--model_type=gpt2 "+\
"--model_name_or_path=distilgpt2 "+\
f"--train_data_file={file_with_training_set} "+\
"--do_train "+\
"--overwrite_output_dir "+\
"--save_steps=500 " +\
f"--num_train_epochs={epochs} \n" +\
"done"

In [ ]:
# Write the fine-tuning bash script to a file called 'run_experiments.sh'
# This will be executed in the next cell to launch the training loops
f = open('run_experiments.sh',mode='w')
f.write(text)
f.close()

This is the core training step where the GPT-2 model learns from our dataset using the training script we generated earlier. This loop runs for 3 epochs, saving checkpoints at defined intervals.

**Note:** Due to time and GPU constraints, I manually interrupted training after about 25 minutes, or 17% of total steps. Despite the early stop, several useful checkpoints (500, 1000, 1500) were created and will be used in the next step.

In [ ]:
# Launch the fine-tuning process
# Note: This was manually stopped after about 25 minutes at iteration 1700+
!bash run_experiments.sh

In [ ]:
# Check the available checkpoints
!ls experiments/epoch_3

## Save the fine-tuned model


In this step, we save our fine-tuned language model using Hugging Face’s `AutoModelForCausalLM` and `AutoTokenizer` classes.

- The model and tokenizer are loaded from the last available checkpoint, or `checkpoint-1500`
- Then we save them to a new folder: `recipe_generation_model`
- Finally, we mount Google Drive and copy the model over for long-term storage and later use

In [ ]:
# Load the model + tokenizer from the last checkpoint
tokenizer = AutoTokenizer.from_pretrained('experiments/epoch_3/checkpoint-1500')
model = AutoModelForCausalLM.from_pretrained('experiments/epoch_3/checkpoint-1500')

# Define a folder name for the saved model
model_path = 'recipe_generation_model'

# And save the model and tokenizer locally
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
# Mount Google Drive so we can export our fine-tuned model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# And copy the saved model to your Google Drive
import shutil
shutil.copytree(model_path, '/content/drive/MyDrive/' + model_path)

## Test the fine-tuned model's recipe writing skills

In [ ]:
# Begin by grabbing the test set
print(dataset_test[0])

In [ ]:
# Need to split at 'Instructions' which will be our prompt
print(dataset_test[0].split('Instructions:')[0] + '\n Instructions')

In [ ]:
# Now store it as input_text_test
input_text_test = dataset_test[0].split('Instructions:')[0] + '\n Instructions'

In [ ]:
input_text_test

Take 2 with the recipe generation...

In [ ]:
# And now to test out the model
input_text_test = dataset_train[0].split('Instructions:')[0]+'Instructions:'
enc_input = tokenizer.encode(input_text_test, return_tensors='pt', add_special_tokens=False)
enc_input
output_sequences = model.generate(
    input_ids = enc_input, # The encoded input from earlier
    max_length= 300,  # The max length of the generated sentence
    temperature = 0.9, # Controls randomness, closer to 1 = more creative, closer to 0 = more predictable
    top_k = 20, # Considers only the top 20 most likely next words
    top_p = 0.9, # Allows dynamic cutoff of word options based on cumulative probabilioty
    repetition_penalty = 1, # Penalty for repeating a word in the input
    do_sample = True, # If true, this allows for randomness instead of always picking the highest-probability word
    num_return_sequences = 5 # Number of output sentences
)
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

And look at that! These instructions actually make sense. You could cook these meals.

Just for fun, let's see if the model can handle that **snowboarding prompt** from before.

In [ ]:
# Let's quickly update the model
input_text_snowboarding = 'I hit the slopes early to snowboard, caught the first lift, and by noon I had already'
enc_input = tokenizer.encode(input_text_snowboarding, return_tensors='pt', add_special_tokens=False)
enc_input
output_sequences = model.generate(
    input_ids = enc_input, # The encoded input from earlier
    max_length= 70,  # The max length of the generated sentence
    temperature = 0.9, # Controls randomness, closer to 1 = more creative, closer to 0 = more predictable
    top_k = 20, # Considers only the top 20 most likely next words
    top_p = 0.9, # Allows dynamic cutoff of word options based on cumulative probabilioty
    repetition_penalty = 1, # Penalty for repeating a word in the input
    do_sample = True, # If true, this allows for randomness instead of always picking the highest-probability word
    num_return_sequences = 5 # Number of output sentences
    )
for i in range(len(output_sequences)):
  print(f'{i}: {tokenizer.decode(output_sequences[i])}\n')

**Final check: model analysis after fine-tuning**

The output above confirms that fine-tuning worked as intended: the model can now generate coherent recipe instructions, but fails to respond meaningfully to off-domain prompts -- like snowboarding. This is a sign of successful domain specialization. Our GPT-2 model has learned to focus on recipe generation, not general language tasks.
